In [ ]:
import importlib

import dataset, metrics, plotting, config, network
from models import cgan_model
import numpy as np
import random

importlib.reload(network)
importlib.reload(dataset)
importlib.reload(metrics)
importlib.reload(plotting)
importlib.reload(config)
importlib.reload(cgan_model)

In [ ]:
import os

dataset_config = config.DatasetConfig(scenario="hdpe")

assert(dataset_config.scenario == "magical_sinus"
      or dataset_config.scenario == "hdpe")
fig_dir = f"../figures/{dataset_config.scenario}"

try:
    os.mkdir(fig_dir)
    print(f"Directory {fig_dir} created ")
except FileExistsError:
    print(f"Directory {fig_dir} already exists replacing files in this notebook")

In [ ]:
exp_config = config.Config(
    model=config.ModelConfig(activation="elu", lr_gen=0.0001, lr_disc=0.0001,
                             optim_gen="Adam", optim_disc="Adam", z_input_size=5),
    training=config.TrainingConfig(n_epochs=10000, batch_size=100, n_samples=500),
    dataset=dataset_config,
    run=config.RunConfig(save_fig=1)
)

In [ ]:
# Set random seed
np.random.seed(exp_config.model.random_seed)
random.seed(exp_config.model.random_seed)

from tensorflow import set_random_seed
set_random_seed(exp_config.model.random_seed)

In [ ]:
X_train, y_train, X_valid, y_valid = dataset.get_dataset(scenario=exp_config.dataset.scenario,
                                                                         seed=exp_config.model.random_seed)

In [ ]:

from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler()
X_train_scaled = X_scaler.fit_transform(X_train)

## Finding sparse regions using Local Outlier Factor (LOF)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor


X = X_train_scaled
clf = LocalOutlierFactor(n_neighbors=40)
clf.fit(X)
X_scores = clf.negative_outlier_factor_
mean_scores = np.mean(X_scores)
mask = X_scores > mean_scores

X_outliers = X[mask]

## CVT sampling and discrepancy evaluating

In [ ]:
from idaes.surrogate.pysmo import sampling as sp
import matplotlib.pyplot as plt
from diversipy import unanchored_L2_discrepancy

assert len(X_outliers) !=0, "No sparse regions in the input space has been found."

min_n_sampling, max_n_sampling, interval = 60, 500, 60

minimum, maximum = np.amin(X_outliers, axis=0).tolist(), np.amax(X_outliers, axis=0).tolist()
bounds = [minimum, maximum]

X_discrepancy = unanchored_L2_discrepancy(X_outliers)
X_CVT_discrepancy_list, X_full_discrepancy_list, CVT_dict = [], [], {}
i = 0
for n_sampling in range(min_n_sampling, max_n_sampling, interval):
    space_init = sp.CVTSampling(bounds, sampling_type="creation", number_of_samples=n_sampling)
    X_CVT = space_init.sample_points()
    CVT_dict[f"num_sampling_{n_sampling}"] = X_CVT
    X_CVT_discrepancy = unanchored_L2_discrepancy(X_CVT)
    X_CVT_discrepancy_list.append(X_CVT_discrepancy)

    X_full = np.r_[X, X_CVT]
    X_full_discrepancy = unanchored_L2_discrepancy(X_full)
    X_full_discrepancy_list.append(X_full_discrepancy)
    print(f"Discrepancy values: {X_discrepancy:.2e} for original data; "
          f"{X_CVT_discrepancy:.2e} for {n_sampling} CVT samples; "
          f"{X_full_discrepancy:.2e} for full data.")

best_n_sampling = np.arange(min_n_sampling, max_n_sampling, interval)[np.argmin(np.array(X_full_discrepancy_list))]
best_CVT = CVT_dict[f"num_sampling_{best_n_sampling}"]
print(f"The  optimal value for n_sampling in CVT sampling is {best_n_sampling}")

In [ ]:
X_discrepancy = unanchored_L2_discrepancy(X_outliers)
plotting.plot_cvt_discrepancy(min_n_sampling, max_n_sampling, interval,
                              X_full_discrepancy_list, fig_dir,
                              title=f"Discrepancy baseline = {X_discrepancy:2.2e}")

In [ ]:
sampling_key = f"num_sampling_{best_n_sampling}"
X_CVT = CVT_dict[sampling_key]

# save CVT samples to .npy file
from os.path import basename
np.save(f"{fig_dir}/{basename(fig_dir)}_CVT_samples.npy", X_scaler.inverse_transform(X_CVT))